# Ipyreact walkthrough

Welcome to this ipyreact walkthrough!   
The tutorial will be based on a very simple react button to show all the ipyreact features.  

**Content** 
* use the %react cell magic
* write a widget
* load CSS for your widget
* add parameters to your widgets (traitlets)
* interact with these parameters
* observe a python function
* observe a javascript function
* enable autocompletion in IDEs
* print a message at class initialization

First, we will use the **`%react` magic**  from ipyreact.  
The following line registers the cellmagic:

In [ ]:
#%pip install ipyreact
# This line is for JupyterLite (in case that this takes long, restart the kernel and run this cell again)

In [ ]:
%load_ext ipyreact

In [ ]:
%%react

import * as React from "react";

export default function MyButton() {
    return ( < button > X < /button>);
    }

This works like a charm!  
Next, we **convert this into a widget.**  
For that, we need the code in a _ems string inside a class that inherits from `ipyreact.ReactWidget`

In [ ]:
import ipyreact

class MyExampleWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function MyButton() {
        return < button > X < /button> 
    };"""

MyExampleWidget()

now it's time to make some **styling using CSS**

In [ ]:
from IPython.display import HTML
css_rules = """
button {
    background-color: yellow;
    border-radius: 10px;
}
"""
HTML("<style>" + css_rules + "</style>")

You can also load CSS from a file.  
And you can add extra CSS without overwriting the previous CSS import.

In [ ]:
from IPython.display import HTML
from pathlib import Path
css_rules = Path("styles_orange.css").read_text()
HTML("<style>" + css_rules + "</style>")

Note: When you clear the output of the previous cell, this CSS style will be removed from the notebook.

Next, we want to **add parameters to the widget.**  
First naive approach: Using f-strings.
That works, but is bad for two reasons:
1. curly brackets from f-strings will collide with the curly brackets from css. 
Using curly brackets in an f-string as a normal character can be done by doubling them like this : `{`to `{{`.
2. It is not possible to pass this parameter into the `MyExampleWidget` class, as it does not have an init function.
TODO: give a reason why there is no normal init function.

In [ ]:
import ipyreact

# ❌❌❌  WARNING: THIS CODEBLOCK IS NOT GOOD PRACTICE ❌❌❌ 
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = "Hello World"
    _esm = f"""
    import * as React from "react";

    export default function MyButton() {{
        return <button> {my_message} </button> 
        }};"""
MyExampleWidget()

Instead, we can **use traitlets.**  
traitlets are variables that can be used both in JavaScript and python.  
Note that it has to be initilized at `MyButton({my_message})`.

In [ ]:
import ipyreact
from traitlets import Unicode

# ⭐⭐⭐  This is good practice again ⭐⭐⭐
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = Unicode("Hi There").tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_message }) {
    return <button> {my_message} </button>;
    };"""
MyExampleWidget()

In [ ]:
# we can use this traitlets also as parameters
MyExampleWidget(my_message = "Super Button")

Now comes an example of a button that changes the postion using traitlets

In [ ]:
import ipyreact
from traitlets import Int

class MyExampleWidget(ipyreact.ReactWidget):
    my_width = Int(23).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
      return (
        <button
          style={{
            position: "relative",
            width: my_width,
            height: 30,
          }}
        >
          {" "}
          Width of {my_width} px{" "}
        </button>
      );
    }"""
MyExampleWidget(my_width=300)

In [ ]:
MyExampleWidget(my_width=600)
#MyExampleWidget(my_width="Hello") # <- this will throw an error, as my_width expects an integer

In [ ]:
# you can also create an instance of a class
w = MyExampleWidget(my_width=600)
w

In [ ]:
# and display this instance multiple times
w

In [ ]:
w.my_width = 400 # see how this will change the button width

In [ ]:
w.my_width = 200 # and one more time

In [ ]:
w.my_width # and get current parameter like this

Next, we will **add a counter** to this widget.

There are two options:
🌱 Firstly, we can define a `my_count` traitlet and for the type we choose an Int traitlet.  
Note that for for every trait <name> there is a `<name> + on_<name>` pair on the react/frontend side.  
That means for `my_count`, there is `on_my_count`.  


In [ ]:
import ipyreact
from traitlets import Int

# 🌱🌱🌱 First option 🌱🌱🌱


class MyCounterWidget(ipyreact.ReactWidget):
    my_count = Int(0).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({my_count, on_my_count}) {
        return <button onClick={() => on_my_count(my_count + 1)}>
            {my_count} times clicked
        </button>
    };"""
MyCounterWidget()

🍁 Secondly, we can use the build in `value` traitlet, which has the type of traitlets.Any type.
And for `value` there is `on_value`.  


In [ ]:
import ipyreact

# 🍁🍁🍁  Second option 🍁🍁🍁


class MyCounterWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function MyButton({value, on_value}) {
        return <button onClick={() => on_value(value + 1)}>
            {value || 0} times clicked
        </button>
    };"""
m = MyCounterWidget()
m

In [ ]:
m.value # <- like this you can access the value from python

In [ ]:
m.value = 20 # <- setting the value also works here.

next, let's **observe a python function** that is called everytime the trailtel value changes.  
In this case it determines if the number is a prime number or not:

In [ ]:
def is_prime_number(n):
    for i in range(2, n):
        if n % i == 0:
            return "No 🧊🧊🧊"
    return "Yes ✅✅✅"

In [ ]:
is_prime_number(8)

In [ ]:
from traitlets import  Any, observe

class OtherCountWidget(ipyreact.ReactWidget):
    prime_message = Any("Click the Button").tag(sync=True)    
    count = Int(0).tag(sync=True)

    @observe('count')
    def _observe_count(self, change):
        self.prime_message = is_prime_number(self.count)
    
    _esm = """
    import * as React from "react";

    export default function({on_count, debug, count, prime_message}) {
        return <div><button onClick={() => on_count(count + 1)}>
            {count} times clicked 
        </button>
        <br/>
        <span> {prime_message} </span>
        </div>
    };"""
ocw = OtherCountWidget()
ocw

one more thing:  
Having **autocompletion in IDEs** is awesome!  
traitlets don't have that by default, but adding a `signature_has_traits` decorator will do the job!

In [ ]:
import ipyreact
from traitlets import Any, Unicode, Int, observe, signature_has_traits

@signature_has_traits
class MyExampleWidget(ipyreact.ReactWidget):
    my_width = Int(23).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
      return (
        <button
          style={{
            position: "relative",
            width: my_width,
            height: 30,
          }}
        >
          {" "}
          Width of {my_width} px{" "}
        </button>
      );
    }"""
MyExampleWidget(my_width=300)

And this screenshots shows that autocompletion works now:  
<img src="autocomplete_screenshot.png" width="400">

Now we want to **print a message at class initialization** that says "Button was initialized with width 300px!"  
That is possible, but not very straight forward, so feel free to skip this example.

In [ ]:
import ipyreact
from traitlets import Int, signature_has_traits

# 🪄🪄🪄 this is an advanced example, feel free to skip 🪄🪄🪄

@signature_has_traits
class MyExampleWidget(ipyreact.ReactWidget):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.print_welcome_message()

    def print_welcome_message(self):
        print(f"Button was initilized with width of {self.my_width}px ")

    my_width = Int(23).tag(sync=True)

    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
    return (
        <button
        style={{
            position: "relative",
            width: my_width,
            height: 30,
        }}
        >
        {" "}
        Width of {my_width} px{" "}
        </button>
    );
    }"""

MyExampleWidget(my_width=200)